In [1]:
import random
import os
import json
import time

In [2]:
MAX_USERS = 99
number_of_slots_in_table = 36
ROULETTE_WHEEL = {'red':[1,3,5,7,9,12,14,16,18,19,21,23,25,27,30,32,34,36],'black':[2,4,6,8,10,11,13,15,17,20,22,24,26,28,29,31,33,35],'green':[0]}
SPLIT_ODDS = 17
STRAIGHT_ODDS = 35
EVEN_ODDS = 1
COLOR_ODDS = 1
MAXIMUM_LOAN_ALLOWED = 10000
casino_user_database_file = "players.txt"
user_database = {}

In [3]:
def newPlayer():
    global user_database
    name = input("Name:")
    money = float(input("amount to deposit:"))
    uniqueID = str(random.randint(0,MAX_USERS))
    while uniqueID in user_database.keys():
          uniqueID = str(random.randint(0,MAX_USERS))
    user_database[uniqueID] = {'name': name, 'balance':money, 'WLR': None, 'expenditure':0, 'loans':0, 'favorite':None, 'winnings':0, 'roulette':0, 'blackjack':0, 'wins':0, 'losses':0}
    print(f"Hey {name}, you have been assigned {uniqueID} as your ID")
    return uniqueID

In [4]:
def blackjack(uid):
    global user_database
    user_database[uid]['blackjack'] += 1
    dCard = False
    uBust = False
    dBust = False
    with open('card_deck.txt', 'r') as f:
        deck_of_cards = json.load(f)
    user_bet = float(input("Enter bet amount ($)"))
    while user_database[uid]['balance'] - user_bet < 0:
        successfully_borrowed = offer_loan(uid)
        if not successfully_borrowed:
            user_in = input("enter lower amount? y/yes")
            if user_in.upper() not in ['YES','Y']:        
                return
        else:
            user_bet = float(input(f"Enter bet amount below ($) {user_database[uid]['balance']}"))
    user_database[uid]['balance'] -= user_bet
    user_database[uid]['expenditure'] += user_bet
    card_list = list(deck_of_cards.keys())
    user_hand = []
    dealer_hand = []
    for _ in range(2):
        pick = random.choice(card_list)
        card_list.remove(pick)
        user_hand.append(pick)
        pick2 = random.choice(card_list)
        card_list.remove(pick2)
        dealer_hand.append(pick2)
    total = print_hands(user_hand, dealer_hand, deck_of_cards, dCard)
    utotal = total[0]
    dtotal = total[1]
    hit = input("Would you like to hit?, 'Y' 'YES'")
    hitCond = ["Y", "YES"]
    while hit.upper() in hitCond and utotal < 21:
        pick = random.choice(card_list)
        card_list.remove(pick)
        user_hand.append(pick)
        total = print_hands(user_hand, dealer_hand, deck_of_cards, dCard)
        utotal = total[0]
        if utotal < 21:
            hit = input("Would you like to hit?, 'Y' 'YES'")
    dCard = True
    if utotal > 21:
        uBust = True
    if not uBust:
        while dtotal < 17:
            pick2 = random.choice(card_list)
            card_list.remove(pick2)
            dealer_hand.append(pick2)
            total = print_hands(user_hand, dealer_hand, deck_of_cards, dCard)
            dtotal = total[1]
    else:
        print("You busted, you lose")
        user_database[uid]['losses'] += 1
    if dtotal > 21:
        print(f"The dealer busted, you win {2 * user_bet}")
        user_database[uid]['balance'] += (2 * user_bet)
        user_database[uid]['wins'] += 1
        user_database[uid]['winnings'] += (2 * user_bet)
    elif dtotal > utotal and not uBust:
        print("The dealer got more that you, you lose")
        user_database[uid]['losses'] += 1
    elif utotal > dtotal:
        print(f"You got more than the dealer, you win {2 * user_bet}")
        user_database[uid]['balance'] += (2 * user_bet)
        user_database[uid]['winnings'] += (2 * user_bet)
        user_database[uid]['wins'] += 1
    else:
        print(f"You tied the dealer, you get {user_bet} back")
        user_database[uid]['balance'] += (user_bet)
        user_database[uid]['winnings'] += (user_bet)

In [5]:
def offer_loan(userID):
    global user_database
    user_in = input("Your bet amount takes your balance below 0. take loan? y/yes to accept")
    valid_input = ['Y','YES']
    if user_in.upper() in valid_input:        
        if user_database[userID]['loans'] >= MAXIMUM_LOAN_ALLOWED: # is user already at limit?
            print(f"Sorry, you are currently at maximum credit. Your outstanding balance is:{user_database[userID]['loans']}")             
            return False        
        else:            
            amount_requested = float(input(f"How much would you like to borrow? max: {MAXIMUM_LOAN_ALLOWED - user_database[userID]['loans']}"))            
            while (amount_requested + user_database[userID]['loans']) > MAXIMUM_LOAN_ALLOWED:                
                amount_requested = float(input(f"Please enter an amount between $0 and {MAXIMUM_LOAN_ALLOWED - user_database[userID]['loans']}"))                                              
            user_database[userID]['loans'] += amount_requested
            user_database[userID]['balance'] += amount_requested
            print(f"You've successfully borrowed {amount_requested}, your current balance is {user_database[userID]['balance']}")   
            return True
    else:
        return False

In [6]:
def print_hands(user_cards, dealer_cards, deck, dealer_card_revealed):
    user_total = 0
    print(f"user cards: ", end = '')
    for card in user_cards:
        user_total += deck[card]
        print(f"{card}", end=' ')
    print (f"total: {user_total}")
    dealer_total = 0
    if dealer_card_revealed:
        print(f"dealer cards: ", end = '')
        for card in dealer_cards:
            dealer_total += deck[card]
            print(f"{card} ", end=' ')
        print (f" total: {dealer_total}")
        return user_total,dealer_total
    else:
        print(f"dealer cards: ", end = '')
        for card in dealer_cards[:-1]:
            dealer_total += deck[card]
            print(f"{card} ", end=' ')
        print("+ face down card")
        print (f" total: {dealer_total}")
    return user_total,dealer_total

In [7]:
def spin_slot_wheel(delay = 1,dots = 3):
    global number_of_slots_in_table
    print("spinning",end='')

    for dot in range(dots):
        time.sleep(delay)
        print('.', end = ' ')
    random_number =  random.randint(0,number_of_slots_in_table)
    color = get_slot_color(random_number, ROULETTE_WHEEL)
    print(f"landed on {color}: {random_number}")
    return random_number, color

In [8]:
def get_slot_color(number, roulette_table):
    color = None
    for col,slot_numbers in roulette_table.items():
        if number in slot_numbers:
            color = col
    return color

In [9]:
def play_roulette(uid):
    global user_database
    user_database[uid]['roulette'] += 1
    user_bet = float(input("Enter bet amount ($)"))
    while user_database[uid]['balance'] - user_bet < 0: # account goes negative
        successfully_borrowed = offer_loan(uid)
        if not successfully_borrowed:
            user_in = input("enter lower amount? y/yes")
            if user_in.upper() not in ['YES','Y']:        
                return
        else:
            user_bet = float(input(f"Enter bet amount below ($) {user_database[uid]['balance']}")) 
            
    user_database[uid]['expenditure'] += user_bet
    user_database[uid]['balance'] -= user_bet # adjust user account balance accordingly
    winner = False
    odds = 0
    bet_type = int(input("What kind of bet would you like to place?\nStraight up betting:0\nsplit betting:1\nColor betting:2\nEven or odd betting:3\n"))
    if bet_type == 0:        
        bet = input("Enter a number to bet on")
        odds = 35
    elif bet_type == 1:
        bet = input("Enter a number to bet on")
        bet2 = input("Enter another number to bet on")
        odds = 17
    elif bet_type == 2:
        bet = input("Pick Black or Red")
        odds = 2
    else:
        bet = input("Pick even or odd\nEven: 0\nOdd: 1\n")
        odds = 2
    resu = spin_slot_wheel()
    num = resu[0]
    color = resu[1]
    if bet_type == 0:
        if num == bet:
            print(f"You won {odds * user_bet}")
            user_database[uid]['balance'] += (odds * user_bet)
            user_database[uid]['wins'] += 1
            user_database[uid]['winnings'] += (odds * user_bet)
        else:
            print("You lost")
            user_database[uid]['losses'] += 1
    elif bet_type == 1:
        if num == bet or num == bet2:
            print(f"You won {odds * user_bet}")
            user_database[uid]['balance'] += (odds * user_bet)
            user_database[uid]['wins'] += 1
            user_database[uid]['winnings'] += (odds * user_bet)
        else:
            print("You lost")
            user_database[uid]['losses'] += 1
    elif bet_type == 2:
        if bet.upper() == color.upper():
            print(f"You won {odds * user_bet}")
            user_database[uid]['balance'] += (odds * user_bet)
            user_database[uid]['wins'] += 1
            user_database[uid]['winnings'] += (odds * user_bet)
        else:
            print("You lost")
            user_database[uid]['losses'] += 1
    elif bet_type == 3:
        if int(num%2) == int(bet):
            print(f"You won {odds * user_bet}")
            user_database[uid]['balance'] += (odds * user_bet)
            user_database[uid]['wins'] += 1
            user_database[uid]['winnings'] += (odds * user_bet)
        else:
            print("You lost")
            user_database[uid]['losses'] += 1

In [10]:
def print_leader_board(score = 'winnings'): # 'winnings' is the default key to sort on if the user doesn't provide a key
    global user_database
    print('*'*32) # print **************************
    # print('*','\t'*2,'*')
    # print('*','\t'*2,'*')
    print('\tLeader board') # \t prints a tab
    print('*'*32) # print **************************
    # sort the players according to the key (score)
    #first, put the usernames and the score keys in different lists
    score_list = []
    user_list = []
    user_id_list = []
    # print(user_database)
    for key, value in user_database.items():
        score_list.append(value[score])
        user_list.append(value['name'])
        user_id_list.append(key)
    # then zip them and sort the user id according to the score
    zipped_values = sorted(zip(score_list,user_id_list), reverse=True)    
    sorted_ids = [userid for score,userid in zipped_values]

    # print(score_list)
    # print(user_list)
    # now, that the user id's are sorted acoording to score,let's print the user id in sequence and whatever else we want tos show
    print (f"\tName\t{score}")
    print('_'*32) # print line
    for id_num in sorted_ids:
        print(f"\t{user_database[id_num]['name']}\t{user_database[id_num][score]}")
    print('_'*32) # print line

In [ ]:
global user_database
files = [item.name for item in os.scandir() if item.is_file()]
if casino_user_database_file in files:      
    with open(casino_user_database_file, 'r') as working_file:
        user_database = json.load(working_file)
else:
    user_database = {}
    newPlayer()
uid = input("Enter user ID to log in: (or \"new\" if new player)")
if uid in user_database:
    print(f"{user_database[uid]['name']}, you can play! balance: {user_database[uid]['balance']}")
else:
    uid = newPlayer()
quitConitions = ["Q", "QUIT", "Y", "YES"]
uInput = "R"
while uInput.upper() not in quitConitions:
    uInput = input("Would you like to quit?, 'Q' or 'Quit'")
    if uInput.upper() not in quitConitions:
        game = int(input("What game would you like to play?:\nBlackJack: 0 \nRoulette: 1\n"))
        if game == 0:
            blackjack(uid)
        elif game == 1:
            play_roulette(uid)
        else:
            print("Not a valid game")
with open (casino_user_database_file, 'w' ) as working_file:
    json.dump(user_database, working_file)
print_leader_board()

Enter user ID to log in: (or "new" if new player) new
Name: h
amount to deposit: 7


Hey h, you have been assigned 83 as your ID


Would you like to quit?, 'Q' or 'Quit' f
What game would you like to play?:
BlackJack: 0 
Roulette: 1
 1
Enter bet amount ($) 6
What kind of bet would you like to place?
Straight up betting:0
split betting:1
Color betting:2
Even or odd betting:3
 0
Enter a number to bet on 7


spinning. . . landed on red: 19
You lost


Would you like to quit?, 'Q' or 'Quit' no
What game would you like to play?:
BlackJack: 0 
Roulette: 1
 0
Enter bet amount ($) 100
Your bet amount takes your balance below 0. take loan? y/yes to accept y
How much would you like to borrow? max: 10000 5000


...

user cards: 8 of Hearts Ace of Hearts total: 9
dealer cards: King of Clubs  + face down card
 total: 10


Would you like to hit?, 'Y' 'YES' y


user cards: 8 of Hearts Ace of Hearts 2 of Clubs total: 11
dealer cards: King of Clubs  + face down card
 total: 10


Would you like to hit?, 'Y' 'YES' y


user cards: 8 of Hearts Ace of Hearts 2 of Clubs 10 of Spades total: 21
dealer cards: King of Clubs  + face down card
 total: 10
user cards: 8 of Hearts Ace of Hearts 2 of Clubs 10 of Spades total: 21
dealer cards: King of Clubs  4 of Spades  King of Spades   total: 24
The dealer busted, you win 800.0


In [150]:
aCount = 0
bCount = 0
cCount = 0
dCount = 0
name = input("What is your name")
q = open("harry_potter_registry", 'r')
harry_people = q.read()
q.close()
harry_people += name
for i in range(9):
    if i != 0:
        R = (6 * (i - 1)) - 1
        N = i * 6
        with open("harry_potter_questions.txt", "r") as file:
            for i in range(N):
                line = next(file).strip()
                if i > R:
                    print(line)
        q = input()
        R = N
        if q.upper() == "A":
            aCount += 1
        elif q.upper() == "B":
            bCount += 1
        elif q.upper() == "C":
            cCount += 1
        elif q.upper() == "D":
            dCount += 1
if aCount > bCount and aCount > cCount and aCount > dCount:
    print("You belong in house Gryffindor")
    harry_people += " You belong in house Gryffindor"
elif bCount > aCount and bCount > cCount and bCount > dCount:
    print("You belong in house Hufflepuff")
    harry_people += " You belong in house Hufflepuff"
elif cCount > aCount and cCount > bCount and cCount > dCount:
    print("You belong in house Ravenclaw")
    harry_people += " You belong in house Ravenclaw"
elif dCount > aCount and dCount > bCount and dCount > cCount:
    print("You belong in house Slytherin")
    harry_people += " You belong in house Slytherin"
else:
    print("Your house cannot be determined")
    harry_people += " Your house cannot be determined"
q = open("harry_potter_registry", 'a')
q.write(harry_people)
q.close()

What is your name Greg


If you could have any pet, what would it be?
a. Owl
b. Cat
c. Toad
d. Snake



 b


Which of the following qualities do you value most in yourself?
a. Bravery
b. Loyalty
c. Intelligence
d. Ambition



 c


...

What music is your favorite
A. Classic Rock
B. EDM
C. County
D. Rap



 c


Who do you aspire to be
A. John F. Kennedy
B. Gandhi
C. Napoleon Bonaparte
D. Stalin



 c


You belong in house Ravenclaw


In [151]:
aCount = 0
bCount = 0
cCount = 0
dCount = 0
name = input("What is your name")
q = open("harry_potter_registry", 'r')
harry_people = q.read()
q.close()
harry_people += name
for i in range(9):
    if i != 0:
        R = (6 * (i - 1)) - 1
        N = i * 6
        with open("harry_potter_questions.txt", "r") as file:
            for i in range(N):
                line = next(file).strip()
                if i > R:
                    print(line)
        q = input()
        R = N
        if q.upper() == "A":
            aCount += 1
        elif q.upper() == "B":
            bCount += 1
        elif q.upper() == "C":
            cCount += 1
        elif q.upper() == "D":
            dCount += 1
if name.upper() == "HARRY POTTER":
    print("You belong in house Slytherin with a 3.9 foot python")
    harry_people += " You belong in house Slytherin with a 3.9 foot python"
elif name.upper() == "HERMIONE":
    print("You belong in house Gryffindor")
    harry_people += " You belong in house Gryffindor"
else:
    if aCount > bCount and aCount > cCount and aCount > dCount:
        print("You belong in house Gryffindor")
        harry_people += " You belong in house Gryffindor"
    elif bCount > aCount and bCount > cCount and bCount > dCount:
        print("You belong in house Hufflepuff")
        harry_people += " You belong in house Hufflepuff"
    elif cCount > aCount and cCount > bCount and cCount > dCount:
        print("You belong in house Ravenclaw")
        harry_people += " You belong in house Ravenclaw"
    elif dCount > aCount and dCount > bCount and dCount > cCount:
        print("You belong in house Slytherin")
        harry_people += " You belong in house Slytherin"
    else:
        print("Your house cannot be determined")
        harry_people += " Your house cannot be determined"
q = open("harry_potter_registry", 'a')
q.write(harry_people)
q.close()

What is your name Byron


If you could have any pet, what would it be?
a. Owl
b. Cat
c. Toad
d. Snake



 d


Which of the following qualities do you value most in yourself?
a. Bravery
b. Loyalty
c. Intelligence
d. Ambition



 b


...

What music is your favorite
A. Classic Rock
B. EDM
C. County
D. Rap



 b


Who do you aspire to be
A. John F. Kennedy
B. Gandhi
C. Napoleon Bonaparte
D. Stalin



 b


You belong in house Hufflepuff


In [3]:
import random
with open("student_names.txt", "r") as file:
    names = [name.strip() for name in file.readlines()]
    
with open("student_emails.txt", "w") as file:
    for name in names:
        first = name.split()[2][0] if len(name.split()) > 2 else ''
        second = name.split()[1][0]
        third = name.split()[0][0]
        num = random.randint(10000, 99999)
        semail = (f"{first}{second}{third}{num}@uga.edu")
        file.write(semail + '\n')

In [40]:
import random
with open("athens_attractions.txt", "r") as file:
    attractions = file.readlines()
    randAttractions = random.sample(attractions, 3)
    attraction1 = randAttractions[0]
    attraction2 = randAttractions[1]
    attraction3 = randAttractions[2]
print(attraction3)
with open("email_template.txt", "r") as file:
    temp = file.read()
with open("student_names.txt", "r") as snames:
    with open("completed_emails.txt", "w") as email:
        for name in snames:
            parts = name.split()
            fname = parts[-1]
            lname = parts[0]
            info = temp.replace('[first_name]', fname).replace('[last_name]', lname).replace('[attraction1]', attraction1).replace('[attraction2]', attraction2).replace('[attraction3]', attraction3).replace('[from_sender]', "Greg Steckel").replace('[sender_title]', "INFO2000 student\n")
            email.write(info)
email.close()

The State Botanical Garden of Georgia - an expansive garden featuring beautiful plants, flowers, and trails that are perfect for a peaceful walk.



In [41]:
def camel_to_snake(inp):
    snake = ''
    for i in inp:
        if i.isupper():
            snake += "_" + i.lower()
        else:
            snake += i
    return snake
print (camel_to_snake("myNameIsGregSteckel"))

my_name_is_greg_steckel
